In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [44]:
train=pd.read_csv('../../data/training_10000.tsv', sep='\t')
test=pd.read_csv('../../data/test_1.tsv', sep='\t')
train['Type']='Train' #Create a flag for Train and Test Data set
test['Type']='Test'
fullData = pd.concat([train,test],axis=0) #Combined both Train and Test Data set

In [45]:
fullData.columns # This will show all the column names
fullData.head(10) # Show first 10 records of dataframe
fullData.describe() #You can look at summary of numerical fields by using describe() function
#train.head(2)

,CASEID,DISYR,AGE,GENDER,RACE,ETHNIC,MARSTAT,EDUC,EMPLOY,DETNLF,...,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,ALCDRUG,DSMCRIT,PSYPROB,HLTHINS,PRIMPAY,ALCOHOL
count,1.199900e+04,11999.0,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,...,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000
mean,2.928142e+05,2011.0,7.227602,1.336195,5.538712,4.686724,0.429869,2.691558,2.833403,-5.173598,...,0.006084,0.003834,0.003417,0.028002,2.120843,-0.215685,-0.333361,-2.642804,-2.564297,0.428452
std,6.435723e+05,0.0,2.626138,0.561131,4.072990,1.534061,3.718920,1.727508,1.640918,5.996202,...,0.077765,0.061800,0.058357,0.164986,0.811896,7.869433,4.292391,6.171367,6.982942,0.494875
min,1.000000e+00,2011.0,2.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-9.000000,-9.000000,-9.000000,-9.000000,0.000000
25%,3.000500e+03,2011.0,5.000000,1.000000,5.000000,5.000000,1.000000,2.000000,3.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,-9.000000,1.000000,-9.000000,-9.000000,0.000000
50%,6.000000e+03,2011.0,7.000000,1.000000,5.000000,5.000000,1.000000,3.000000,3.000000,-9.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,2.000000,1.000000,-9.000000,0.000000
75%,8.999500e+03,2011.0,9.000000,2.000000,5.000000,5.000000,2.000000,3.000000,4.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,6.000000,2.000000,4.000000,5.000000,1.000000
max,1.732741e+06,2011.0,12.000000,2.000000,23.000000,6.000000,4.000000,5.000000,4.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,3.000000,20.000000,2.000000,4.000000,9.000000,1.000000


In [98]:
ID_col = ['CASEID']
target_col = ["BENZFLG"]
cat_cols = ['AGE','GENDER','RACE','ETHNIC','MARSTAT','EDUC','EMPLOY','DETNLF','PREG','VET','LIVARAG','PRIMINC','ARRESTS','STFIPS','CBSA','PMSA','REGION','DIVISION','SERVSETD','METHUSE','DAYWAIT','REASON','LOS','PSOURCE','DETCRIM','NOPRIOR','ROUTE1','FREQ1','FRSTUSE1','ROUTE2','FREQ2','FRSTUSE2','ROUTE3','FREQ3','FRSTUSE3','NUMSUBS','IDU']
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(ID_col)-set(target_col))
other_col=['Type'] #Test and Train Data set identifier

In [99]:
num_cat_cols = num_cols+cat_cols # Combined numerical and Categorical variables

#Create a new variable for each variable having missing value with VariableName_NA 
# and flag missing value with 1 and other with 0

for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1 
        
        
        

In [100]:
# fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean(),inplace=True)

In [101]:
#Impute categorical missing values with -9999
# fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)

In [102]:
# #create label encoders for categorical features
# for var in cat_cols:
#  number = LabelEncoder()
#  fullData[var] = number.fit_transform(fullData[var].astype('str'))

In [103]:

#Target variable is also a categorical so convert it
# fullData["ALCOHOL"] = number.fit_transform(fullData["ALCOHOL"].astype('str'))

train=fullData[fullData['Type']=='Train']
test=fullData[fullData['Type']=='Test']


In [104]:
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]

/Users/apoorva/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [105]:
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(other_col) - set(num_cols))

In [106]:
x_train = Train[list(features)].values
y_train = Train["BENZFLG"].values
x_validate = Validate[list(features)].values
y_validate = Validate["BENZFLG"].values
x_test=test[list(features)].values


In [114]:
random.seed(100)
rf = GradientBoostingClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

y_test=rf.predict(x_test)

In [115]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, test[target_col])

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.20


In [38]:
from sklearn.metrics import roc_curve, auc
status = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, status[:,1])
roc_auc = auc(fpr, tpr)
print roc_auc

final_status = rf.predict_proba(x_test)
test["COKEFLG"]=final_status[:,1]
test.to_csv('../../data/output.csv',columns=['CASEID','COKEFLG'])

1.0


/Users/apoorva/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
